## Введение в обучение нейронных сетей с Keras

Данный пример посвящен обучению сверточной нейронной сети на наборе MNIST.    
Более подробно о том как работают сверточные нейронные сети можно прочесть здесь: http://cs231n.github.io/convolutional-networks/

In [0]:
# Для Google Colab - Устанавливаем последнюю версию tensorflow и keras
! pip install tensorflow keras --upgrade

In [1]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.0.0
2.3.1


Using TensorFlow backend.


In [0]:
# coding=utf-8
#from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

nb_classes = 10
# размер изображения
img_rows, img_cols = 28, 28

# данные были перемешаны и разделены на обучающий и проверочный наборы
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#изменяем размер массива  (кол-во примеров, w,h, кол-во каналов изображения)
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

#переводим значения в диапазон от 0 до 1
X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# преобразование вектора разметки в матрицу кол-во примеров*кол-во классов
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

Далее создадим модель сверточной нейронной сети.    
Обратите внимание как задаются параметры входных данных для первого слоя: input_shape=(28, 28, 1) - (w,h, кол-во каналов изображения). Такой способ задания называется channels_last и применяется в tensorflow. В Theano используется формат (channels, w, h), из-за этих различий в строке "image_dim_ordering" конфигурационного файла Keras необходимо указывать используемый формат.    
По этой причине код сверточных сетей не будет запускаться без правок с раными бэкендами.

In [0]:
model = Sequential()

#32 фильтра размером (3,3), 
#padding='valid' - не добавляем 0 на границе изображения, 
#после свертки получим меньшую размерность 26*26
#padding='same' - добавляем 0, после свертки размерность будет 28x28
model.add(Conv2D(32, (3, 3), padding='valid', input_shape=(28, 28, 1)))
#params: 32*(3x3+1) = 320, output: (28-1-1)x(28-1-1) x 32 = 26x26 x 32
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3), activation ='relu'))
#params: 32*(32*3*3+1) = 9248, output (26-2)x(26-2) x 32 = 24x24 x 32
#model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
#output: 12x12 x 32

model.add(Dropout(0.25))

model.add(Flatten())
#input: 12x12 x 32, output: 12x12x32=4608

model.add(Dense(128))
#params: (4608+1)*128=589952, output: 128
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(nb_classes))
#params: (128+1)*10=1290

model.add(Activation('softmax'))

#Print model info
#https://keras.io/models/about-keras-models/
model.summary()

#Total params: 600810

model.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=128, epochs=1, #5,12
verbose=1, validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test,verbose=0)
print('Test accuracy:', score[1])

sudo apt-get install libhdf5-dev  
pip install h5py

In [0]:
#Сохранение обученной модели на диск
model.save('cnn.h5')

In [0]:
del model

# загрузка модели
model = load_model('cnn.h5')

In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

sample_idx = np.random.randint(X_train.shape[0])  # выберем случайное изображение
sample_image = X_train[sample_idx, :, :,0]  # extract a 2D slice 

plt.figure()
plt.imshow(sample_image, cmap='gray')  # и выведем его на экран

#Передаем изображение сети
print np.argmax(model.predict(X_train[sample_idx,:].reshape(1, img_rows, img_cols, 1)))

In [0]:
from keras.preprocessing.image import img_to_array, load_img

#чтение изображения в массив
image = load_img('/media/a/test.jpg')
image = img_to_array(image)



### Вопросы

1. Выполните обучение сети с алгоритмами оптимизации sgd (стохастический градиентный спуск) и adam. Сравните точность распознавания сверточной и полносвязной нейронных сетей. https://keras.io/optimizers/ 
Более подробно чем отличаются алгоритмы оптимизации друг от друга: https://habrahabr.ru/post/318970/ 
2. Сравните результаты обучения с разным количеством эпох.
3. Опишите какие операции выполняются на каждом из слоев сети.
4. Напишите цифру на изображении 28*28 и проверьте справится ли с ним сеть.